In [21]:
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer
from xgboost.sklearn import XGBRegressor
from matplotlib import pyplot
import matplotlib.pyplot as plt
from bayes_opt import BayesianOptimization

# data_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__file__))), 'user_data')
# model_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'models')
# output_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__file__))), 'prediction_result')

# data_path = os.path.join(data_dir, 'train_tree.csv')
# test_data_path = os.path.join(data_dir, 'test_tree.csv')
# model_path = os.path.join(model_dir, 'xgb.pkl')
# output_path = os.path.join(output_dir, 'xgb.csv')
data_path = './user_data/train_tree.csv'
test_data_path = './user_data/test_tree.csv'
model_path = './model/xgb.pkl'
output_path = './prediction_result/xgb.csv'

In [22]:
df_train = pd.read_csv(data_path)
df_test = pd.read_csv(test_data_path)

In [23]:
print(df_train.shape)
print(df_test.shape)

(143636, 595)
(50000, 594)


In [24]:
df_train.head()

,name,model,brand,power,kilometer,regionCode,price,v_0,v_1,v_2,...,v_12_multiply_v_13,v_12_div_v_13,v_12_add_v_14,v_12_minus_v_14,v_12_multiply_v_14,v_12_div_v_14,v_13_add_v_14,v_13_minus_v_14,v_13_multiply_v_14,v_13_div_v_14
0,7.504,7.35,7.547,60,12.5,8.450,7.523,43.34,3.967,0.05026,...,-1.925,-3.043940,-1.5060,-3.33600,-2.2150,-2.646393,1.7100,-0.11945,0.7275,0.869397
1,7.727,8.38,8.620,0,15.0,8.805,8.190,45.30,5.234,0.13800,...,1.775,0.598188,-0.7850,-1.27600,-0.2530,-4.197103,-1.4780,-1.96800,-0.4229,-7.016361
2,8.050,9.15,9.040,163,12.5,8.375,8.734,45.97,4.824,1.31900,...,-1.304,-1.879853,1.3350,1.79500,-0.3599,-6.806880,-1.0625,-0.60250,0.1915,3.620964
3,8.190,8.96,8.520,193,15.0,8.080,7.785,45.70,4.492,-0.05063,...,1.224,0.205823,-0.9805,-0.02316,0.2402,1.048399,-2.9180,-1.96000,1.1670,5.093704
4,8.050,6.93,7.688,68,5.0,9.125,8.555,44.38,2.031,0.57230,...,2.639,0.328490,2.8550,-0.99200,1.7910,0.484075,4.7580,0.91100,5.4530,1.473639


In [25]:
y = df_train.pop('price').values
X = df_train.values

In [19]:
def my_score(y_predict, y_true):
    '''
    自定义评估指标
    :return:
    '''
    print(y_true)
    print('===')
    print(y_predict)
    
    score = - mean_absolute_error(np.exp(y_true), np.nan_to_num(np.exp(y_predict)))
#     return 'exp_score', score
#     label = y_predict.get_label()
#     score = - mean_absolute_error(np.exp(label), np.exp(preds))
    return 'exp_score', score
# score = make_scorer(my_score,greater_is_better=True)

In [26]:
def log_transfer(func):  #定义一个将数据转为log的闭包函数
    def wrapper(y, yhat):
        print(y)
        print('='*100)
        print(yhat)
        result = func(np.exp(y), np.nan_to_num(np.exp(yhat)))
        return result
    return wrapper

In [27]:
def xgb_cv(n_estimators, learning_rate, gamma, min_child_weight, max_depth, colsample_bytree, subsample):
    param = {
        'objective': 'reg:squarederror',
        'matric': my_score,
        'random_state': 2020,
        "early_stopping_rounds": 50,
        #         "tree_method": "gpu_hist",
        #         "gpu_id": 0
    }
    param['n_estimators'] = int(n_estimators)
#     param['learning_rate'] = float(learning_rate)
#     param['gamma'] = float(gamma)
#     param['min_child_weight'] = float(min_child_weight)
#     param['max_depth'] = int(max_depth)
#     param['colsample_bytree'] = float(colsample_bytree)
    val = cross_val_score(XGBRegressor(**param),
                          X, y, scoring=make_scorer(log_transfer(mean_absolute_error)), cv=2).mean()
    return val


xgb_bo = BayesianOptimization(
    xgb_cv,
    {'n_estimators': (10, 100),
     'learning_rate': (0.03, 0.3),
     'gamma': (0, 0.5),
     'min_child_weight': (10, 200),
     'max_depth': (4, 10),
     'colsample_bytree': (0.1, 1),
     'subsample': (0.5, 1),
     }
)
xgb_bo.maximize()

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
[7.523 8.19  8.734 ... 7.41  6.746 8.1  ]
[7.6312566 7.5287952 8.688154  ... 7.368878  6.8022404 7.016978 ]
[9.734 8.945 9.58  ... 8.92  8.516 8.45 ]
[9.778195 8.885736 9.446625 ... 8.903965 8.554507 8.452547]
|  1        |  590.6    |  0.3231   |  0.1612   |  0.1591   |  4.585    |  169.4    |  90.01    |  0.9212   |


KeyboardInterrupt: 

In [ ]:
xgb_bo.max['params']

In [ ]:
X